In [41]:
# Import important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

### Q1 Read the input file and check the data dimension

In [42]:
#Read input file and understand the data
# "default" is my dependent variable

In [43]:
data=pd.read_csv('german_credit.csv')
data.head()


,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,...,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,...,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,...,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,...,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes


In [0]:
# Lets build a Ensemble model but need to modify the dataset first


### Q2.Prepare the model data by converting non-numeric to dummy ( 1 Marks)
##### Hint: Use get_dummies

In [44]:
# Print Shape of model data
data.shape


(1000, 21)

In [45]:
data=pd.get_dummies(data=data)

### Check for highly correlated variables but don't required any treatment for this use case

In [46]:
data.shape


(1000, 62)

In [47]:
data.head()

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status_0 <= ... < 200 DM,account_check_status_< 0 DM,...,housing_own,housing_rent,job_management/ self-employed/ highly qualified employee/ officer,job_skilled employee / official,job_unemployed/ unskilled - non-resident,job_unskilled - resident,telephone_none,"telephone_yes, registered under the customers name",foreign_worker_no,foreign_worker_yes
0,0,6,1169,4,4,67,2,1,0,1,...,1,0,0,1,0,0,0,1,0,1
1,1,48,5951,2,2,22,1,1,1,0,...,1,0,0,1,0,0,1,0,0,1
2,0,12,2096,2,3,49,1,2,0,0,...,1,0,0,0,0,1,1,0,0,1
3,0,42,7882,2,4,45,1,2,0,1,...,0,0,0,1,0,0,1,0,0,1
4,1,24,4870,3,4,53,2,2,0,1,...,0,0,0,1,0,0,1,0,0,1


### Drop the original variables which are converted to dummy

In [48]:
#The pd.get_dummies has already dropped all the original variables converted to dummy as shown below
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 62 columns):
default                                                                       1000 non-null int64
duration_in_month                                                             1000 non-null int64
credit_amount                                                                 1000 non-null int64
installment_as_income_perc                                                    1000 non-null int64
present_res_since                                                             1000 non-null int64
age                                                                           1000 non-null int64
credits_this_bank                                                             1000 non-null int64
people_under_maintenance                                                      1000 non-null int64
account_check_status_0 <= ... < 200 DM                                        1000 non-null uint8
account_chec

### Q3 Split Train/Test data 70:30 ratio( 1 Marks)
##### Hint:from sklearn.model_selection import train_test_split

In [50]:
X =  data.drop("default", axis=1)
y= pd.DataFrame(data["default"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2)

### Q4 Build Random Forest Model( 1 Marks)
#### Hint:from sklearn.ensemble import RandomForestClassifier using n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999

In [51]:
rfcl = RandomForestClassifier(n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999)
rfcl.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=9999, verbose=0,
            warm_start=False)

### Q5 Calculate Confusion Matrix and Accuracy score (1 Marks)
##### Hint: Use confusion_matrix and accuracy_score

In [52]:
ypred=rfcl.predict(X_test)
acc=metrics.accuracy_score(y_test,ypred)
print(acc)

0.74


In [53]:
#Confusion matrix

cm=metrics.confusion_matrix(y_test,ypred)
print("Confusion Matrix: \n {}".format(cm))

Confusion Matrix: 
 [[189  12]
 [ 66  33]]


### Q6 Show the list of the features importance( 1 Marks)

In [54]:
print (pd.DataFrame(rfcl.feature_importances_, columns = ["Imp"], index = X_test.columns))


                                                         Imp
duration_in_month                                   0.070840
credit_amount                                       0.098386
installment_as_income_perc                          0.036559
present_res_since                                   0.033072
age                                                 0.080612
credits_this_bank                                   0.018161
people_under_maintenance                            0.012248
account_check_status_0 <= ... < 200 DM              0.017502
account_check_status_< 0 DM                         0.031438
account_check_status_>= 200 DM / salary assignm...  0.009706
account_check_status_no checking account            0.043726
credit_history_all credits at this bank paid ba...  0.007381
credit_history_critical account/ other credits ...  0.020425
credit_history_delay in paying off in the past      0.010232
credit_history_existing credits paid back duly ...  0.013798
credit_history_no credit

### Q7 K-fold cross-validation( 2 Marks)
##### k-fold cross validation( without stratification)
##### Usually k is set as 10-20 in practical settings, depends on data set size

In [55]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [56]:
# Use below values
num_folds = 10
seed = 77

In [26]:
#Validate the Random Forest model build above using k fold

In [57]:
#Validate the Random Forest model build above using k fold
#kf = KFold(X_train.shape[0], n_splits=10,random_state=77)
kf = KFold(n_splits=num_folds,random_state=seed)

# perform 10-fold cross validation
scores = cross_val_score(rfcl, X_train, y_train, cv=kf, scoring='accuracy')

In [60]:
#Calculate Mean score
print("Mean score : %s"%(scores.mean()))

Mean score : 0.78


In [59]:
# Calculate score standard deviation using std()
scores.std()

0.04435478484645722

# Q8 Print the confusion matrix( 1 Marks)

In [61]:
ypred=rfcl.predict(X_test)
cm=metrics.confusion_matrix(y_test,ypred)
print(cm)

[[189  12]
 [ 66  33]]


# Q9.Classification accuracy: 
percentage of correct predictions and Calculate sensitivity (or True Positive Rate or Recall) and Precision.
(2 Marks)

In [62]:
class_acc=metrics.classification_report(y_test,ypred)
print(class_acc)

             precision    recall  f1-score   support

          0       0.74      0.94      0.83       201
          1       0.73      0.33      0.46        99

avg / total       0.74      0.74      0.71       300



[link text](https://)### Bootstrapping ( Bonus and Optional)
##### Given a dataset of size n, a bootstrap sample is created by sampling n instances uniformly from the data (with/without replacement)
##### Create a model with each bootstrap sample and validate it with the test set
##### Final result is calculated by averaging the accuracy of models

In [63]:
# Number of iterations for bootstrapping
bootstrap_iteration = 10
accuracy = []

In [64]:

for i in range(0,bootstrap_iteration):
    #print(i)
    sample=data.sample(500)
    X =  sample.drop("default", axis=1)
    y= pd.DataFrame(sample.pop("default"))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2)
    rfcl = RandomForestClassifier(n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999)
    rfcl.fit(X_train,y_train)
    ypred=rfcl.predict(X_test)
    acc=metrics.accuracy_score(y_test,ypred)
    print(acc)
    accuracy.append(acc)
    
    #sample_data=data.sample(500)
print("The average score is",sum(accuracy)/len(accuracy))

0.72
0.7533333333333333
0.7533333333333333
0.7266666666666667
0.7533333333333333
0.74
0.7333333333333333
0.7666666666666667
0.7266666666666667
0.7533333333333333
The average score is 0.7426666666666666


In [65]:
accuracy = np.array(accuracy)
print('Accuracy Score')
print('Avearge: ', accuracy.mean())
print('Standard deviation: ', accuracy.std())

Accuracy Score
Avearge:  0.7426666666666666
Standard deviation:  0.014666666666666668
